In [2]:
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from  collections import Counter

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'
preprocess_path = 'preprocess'

In [3]:
clk_train_file= 'click_log.pkl'
clk_train_file_path = f'{preprocess_path}/{clk_train_file}'
if not os.path.exists(clk_train_file_path):
    clk_df = pd.read_csv(f'{data_path}/train_preliminary/click_log.csv' )
    clk_df.to_pickle(clk_train_file_path)
else: 
    clk_df = pd.read_pickle(clk_train_file_path)
    
print(clk_df)

          time  user_id  creative_id  click_times
0            9    30920       567330            1
1           65    30920      3072255            1
2           56    30920      2361327            1
...        ...      ...          ...          ...
30082768    26    30920       629802            1
30082769    86    30920      2713031            1
30082770    25    30920       629802            1

[30082771 rows x 4 columns]


In [14]:
user_click_sum_df = clk_df.groupby(['user_id']).click_times.sum().reset_index().rename(columns={'click_times':'click_times_total'})
print(user_click_sum_df)

        user_id  click_times_total
0             1                 14
1             2                 46
2             3                 30
...         ...                ...
899997   899998                 15
899998   899999                 22
899999   900000                 12

[900000 rows x 2 columns]


In [16]:
user_day_count_df = clk_df.groupby(['user_id']).time.nunique().reset_index().rename(columns={'time':'active_days'})
print(user_day_count_df)

        user_id  active_days
0             1           10
1             2           28
2             3           23
...         ...          ...
899997   899998           10
899998   899999           17
899999   900000           12

[900000 rows x 2 columns]


In [19]:
user_log_day_clicks_df = clk_df.groupby(['user_id','time']).creative_id.count().reset_index().rename(columns={'creative_id':'day_clicks'})
print(user_log_day_clicks_df)
user_day_clicks_max_df = user_log_day_clicks_df.groupby(['user_id']).day_clicks.max().reset_index().rename(columns={'day_clicks':'day_clicks_max'})
user_day_clicks_min_df = user_log_day_clicks_df.groupby(['user_id']).day_clicks.min().reset_index().rename(columns={'day_clicks':'day_clicks_min'})
user_day_clicks_mean_df = user_log_day_clicks_df.groupby(['user_id']).day_clicks.mean().reset_index().rename(columns={'day_clicks':'day_clicks_mean'})
user_day_clicks_std_df = user_log_day_clicks_df.groupby(['user_id']).day_clicks.std().reset_index().rename(columns={'day_clicks':'day_clicks_std'})



          user_id  time  day_clicks
0               1    20           3
1               1    39           1
2               1    40           1
...           ...   ...         ...
19825288   900000    72           1
19825289   900000    74           1
19825290   900000    91           1

[19825291 rows x 3 columns]


In [20]:
user_df = user_day_count_df.merge(user_click_sum_df,on='user_id')
user_df = user_df.merge(user_day_clicks_max_df,on='user_id')
user_df = user_df.merge(user_day_clicks_min_df,on='user_id')
user_df = user_df.merge(user_day_clicks_mean_df,on='user_id')
user_df = user_df.merge(user_day_clicks_std_df,on='user_id')
print(user_df)


        user_id  active_days  click_times_total  day_clicks_max  day_clicks_min  day_clicks_mean  day_clicks_std
0             1           10                 14               3               1         1.300000        0.674949
1             2           28                 46               4               1         1.607143        0.994030
2             3           23                 30               3               1         1.304348        0.634950
...         ...          ...                ...             ...             ...              ...             ...
899997   899998           10                 15               3               1         1.400000        0.699206
899998   899999           17                 22               2               1         1.294118        0.469668
899999   900000           12                 12               1               1         1.000000        0.000000

[900000 rows x 7 columns]


In [6]:
ad_df = pd.read_csv(f'{data_path}/train_preliminary/ad.csv' )

In [7]:
merged_df = clk_df.merge(ad_df,on='creative_id')
merged_df.replace("\\N",-1,inplace=True)
merged_df = merged_df.fillna(-1).astype(int)

train_merged= 'train_merged.pkl'
train_merged_path = f'{preprocess_path}/{train_merged}'
merged_df.to_pickle(train_merged_path)

del clk_df

In [8]:
clk_test_file= 'click_log_test.pkl'
clk_test_file_path = f'{preprocess_path}/{clk_test_file}'
if not os.path.exists(clk_test_file_path):
    clk_test_df = pd.read_csv(f'{data_path}/test/click_log.csv' )
    clk_test_df.to_pickle(clk_test_file_path)
else: 
    clk_test_df = pd.read_pickle(clk_test_file_path)
print(clk_test_df)

          time  user_id  creative_id  click_times
0           20  3131989       645764            1
1           20  3131989      1027422            1
2           20  3131989      1106443            1
...        ...      ...          ...          ...
33585509    12  3131989       589027            1
33585510    61  3131989      2404831            1
33585511    14  3131989       665822            1

[33585512 rows x 4 columns]


In [9]:
user_click_sum_test_df = clk_test_df.groupby(['user_id']).click_times.sum().reset_index().rename(columns={'click_times':'click_times_total'})
print(user_click_sum_test_df)

        user_id  click_times_total
0       3000001                 11
1       3000002                 65
2       3000003                 36
...         ...                ...
999997  3999998                 20
999998  3999999                 86
999999  4000000                 67

[1000000 rows x 2 columns]


In [10]:
user_day_count_test_df = clk_test_df.groupby(['user_id']).time.nunique().reset_index().rename(columns={'time':'active_days'})
print(user_day_count_test_df)

        user_id  active_days
0       3000001            7
1       3000002           37
2       3000003           23
...         ...          ...
999997  3999998           18
999998  3999999           45
999999  4000000           39

[1000000 rows x 2 columns]


In [1]:
user_day_clicks_test_df = clk_test_df.groupby(['user_id','time']).industy.count().reset_index().rename(columns={'industy':'day_clicks'})
print(user_day_count_test_df)

NameError: name 'clk_test_df' is not defined

In [11]:
user_test_df = user_day_count_test_df.merge(user_click_sum_test_df,on='user_id')
print(user_test_df)

        user_id  active_days  click_times_total
0       3000001            7                 11
1       3000002           37                 65
2       3000003           23                 36
...         ...          ...                ...
999997  3999998           18                 20
999998  3999999           45                 86
999999  4000000           39                 67

[1000000 rows x 3 columns]


In [12]:
user_total_df = pd.concat([user_df, user_test_df], axis=0).reset_index().drop(['index'], axis=1)
user_total= 'user_total_df.pkl'
user_total_path = f'{preprocess_path}/{user_total}'
user_total_df.to_pickle(user_total_path)

In [13]:
ad_test_df = pd.read_csv(f'{data_path}/test/ad.csv' )
print(ad_test_df)

         creative_id    ad_id product_id  product_category  advertiser_id industry
0                  1        1         \N                 5            381       78
1                  3        3         \N                 5            598      165
2                  4        4         \N                 5            108      202
...              ...      ...        ...               ...            ...      ...
2618156      4445718  3812200      39287                17            427       \N
2618157      4445719  3812201      39287                17            427       \N
2618158      4445720  3812202      39287                17            427       \N

[2618159 rows x 6 columns]


In [14]:
merged_test_df = clk_test_df.merge(ad_test_df,on='creative_id')
merged_test_df.replace("\\N",-1,inplace=True)
merged_test_df = merged_test_df.fillna(-1).astype(int)
print(merged_test_df)

test_merged= 'test_merged.pkl'
test_merged_path = f'{preprocess_path}/{test_merged}'
merged_test_df.to_pickle(test_merged_path)

del clk_test_df

          time  user_id  creative_id  click_times   ad_id  product_id  product_category  advertiser_id  industry
0           20  3131989       645764            1  573314          58                 2          14689         6
1           10  3142948       645764            1  573314          58                 2          14689         6
2           14  3170643       645764            1  573314          58                 2          14689         6
...        ...      ...          ...          ...     ...         ...               ...            ...       ...
33585509     5  3131984       322785            1  290114        2031                 2          19976       238
33585510    16  3131984       900585            1  793805        1766                 2          33097       319
33585511    16  3131988       899450            1  792851          -1                18          22897        47

[33585512 rows x 9 columns]


In [18]:
total_merged_df = pd.concat([merged_df, merged_test_df], axis=0).reset_index().drop(['index'], axis=1)
total_merged_df=total_merged_df.sort_values(by='time')
print(total_merged_df)

del merged_df
del merged_test_df

total_merged= 'total_merged.pkl'
total_merged_path = f'{preprocess_path}/{total_merged}'
total_merged_df.to_pickle(total_merged_path)

In [19]:
def w2v(log,pivot,f,flag,L,model_path,seq_len=200,sentence_len=100,window=5):
    #word2vec算法
    #log为曝光日志，以pivot为主键，f为embedding的对象，flag为dev或test，L是embedding的维度
    print("w2v:",pivot,f,model_path)
    #构造文档
    sentence=[]
    dic={}
    day=0
    log['day'] = log['time']
    print('build...')
    for item in tqdm(log[['day',pivot,f]].values,total=len(log)):
        try:
            user_values = dic[item[1]]
            user_list = user_values[0] + [str(int(item[2]))]
            user_values[1].update([str(int(item[2]))])
            dic[item[1]] = [user_list,user_values[1],user_values[2] +1]
        except:
            c=  Counter([str(int(item[2]))])
            dic[item[1]]=[[str(int(item[2]))],c,1]

    c = Counter()
    for key in dic:
        c.update([len(dic[key][0])])
        sentence.append(dic[key][0])
    print(f"sentence counter:{c}")
    print(len(sentence))
    #训练Word2Vec模型
    print('shuffle...')
    random.shuffle(sentence)
    print('training...')
    model = Word2Vec(sentence, size=L, window=window, min_count=1, workers=10,iter=10)
    print(model_path)
    model.save(model_path)
    del sentence
    print(model)
    
    ret = []
    for key in dic:
         ret.append([key,dic[key][0]])
    print(ret[:3])
    cols = ['user_id'] + [f'{f}_seq']
    ret_df = pd.DataFrame(ret,columns=cols)
    
    del model
    del dic
    del ret
    return ret_df
    


In [21]:
industy_seq_df =  w2v(total_merged_df,'user_id','industry','train',16,'/data/workspace/kimi/tencent_ads/2020/kimi/industry_emb_model',500,500,10)


w2v: user_id industry /data/workspace/kimi/tencent_ads/2020/kimi/industry_emb_model
build...


100%|██████████| 63668283/63668283 [17:55<00:00, 59192.32it/s]  


sentence counter:Counter({11: 93286, 12: 87214, 13: 81029, 10: 77471, 14: 76229, 15: 70573, 16: 66238, 17: 62740, 18: 57944, 19: 55232, 20: 52154, 21: 48979, 22: 46991, 23: 43486, 24: 41647, 25: 39427, 26: 37492, 27: 35835, 28: 34030, 29: 32371, 30: 30529, 31: 29284, 32: 27737, 33: 26608, 34: 25306, 35: 23813, 36: 22840, 9: 22055, 37: 21589, 38: 20817, 39: 19824, 40: 18832, 41: 17805, 42: 16972, 43: 16152, 44: 15485, 45: 14859, 46: 14021, 47: 13440, 48: 12881, 49: 12346, 50: 11843, 51: 10976, 52: 10637, 53: 10448, 54: 9820, 55: 9333, 56: 9043, 57: 8753, 58: 8157, 59: 8077, 60: 7668, 61: 7403, 62: 7140, 63: 6954, 64: 6448, 65: 6262, 66: 6033, 67: 5835, 68: 5552, 69: 5301, 70: 5197, 71: 4957, 72: 4824, 8: 4776, 74: 4573, 73: 4404, 75: 4318, 76: 4079, 77: 3945, 78: 3828, 79: 3707, 80: 3503, 81: 3424, 82: 3370, 83: 3307, 84: 3072, 85: 3062, 86: 2866, 87: 2828, 88: 2781, 89: 2672, 90: 2539, 91: 2440, 92: 2437, 94: 2303, 93: 2298, 95: 2197, 96: 2041, 98: 2003, 97: 1999, 100: 1890, 101: 1863,

training...
/data/workspace/kimi/tencent_ads/2020/kimi/industry_emb_model
Word2Vec(vocab=332, size=16, alpha=0.025)


ValueError: Shape of passed values is (2, 1), indices imply (2, 2)

In [ ]:
advertiser_id_seq_df = w2v(total_merged_df,'user_id','advertiser_id','train',32,'/data/workspace/kimi/tencent_ads/2020/kimi/advertiser_id_emb_model',500,500,10)


In [ ]:
total_final = 'total_final.pkl'
total_final_path = f'{preprocess_path}/{total_final}'


total_df =  user_final_df.merge(advertiser_id_seq_df,on='user_id')
total_df  = total_df.merge(industy_seq_df,on='user_id')
total_df.to_pickle(total_merged_path)